### Classification of AD vs MCI vs NC Using the ResNet Pre-Trained Model

Import the packages

In [ ]:
! pip install -r requirements.txt

In [2]:
! pip install monai

   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   -------------------- ------------------- 0.8/1.5 MB 5.6 MB/s eta 0:00:01
   ---------------------------------------- 1.5/1.5 MB 4.2 MB/s eta 0:00:00


In [3]:
import os
import sys
import numpy as np
import pandas as pd
import torch
from torch import nn
from torchvision import transforms
import matplotlib.pyplot as plt
from PIL import Image
from torch.utils.data import DataLoader
from torchinfo import summary
import shutil
import data_manager as DM
#import torchvision.models as models
import torchvision.models.video as models
from torchvision.models.video import R3D_18_Weights
import data_setup, engine
from helper_functions import plot_loss_curves
from data_setup import create_dataloaders
import engine
from monai.transforms import Resize
from torchvision.transforms.functional import InterpolationMode
from torch.utils.data import DataLoader

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cuda


Add the parrent path to current path because data is there

In [24]:
current_path = os.getcwd()
parrent_path = os.path.abspath(os.path.join(current_path, '..'))
grand_parrent_path = os.path.abspath(os.path.join(current_path, '../..'))
sys.path.append(parrent_path)
sys.path.append(grand_parrent_path)

from Update_Git import git_add, git_commit, git_push
file_path = os.path.join('main.ipynb')
# file_path = os.path.join('data_manager.py')
# file_path = os.path.join('data_setup.py')
# file_path = os.path.join('engine.py')
# file_path = os.path.join('helper_functions.py')
# file_path = os.path.join('model_builder.py')
# file_path = os.path.join('predictions.py')
# file_path = os.path.join('requirements.txt')

git_add(file_path)
git_commit('Updated dahsbord ResNet3D: main')
git_push('main')

KeyboardInterrupt: 

Manage data:

✔ Read subject IDs from each sheet in Subject list.xlsx.

✔ Create Data/AD, Data/MCI, Data/NC folders.

✔ Find std_T1.nii for each subject inside ADNI/{subject_id}/.

✔ Copy & renames the file to Data/{category}/{subject_id}.nii.

In [ ]:
excel_file = "../Subject list.xlsx"
source_root = "ADNI"
destination_root = "Data"
categories = ["AD", "MCI", "NC"]

DM.copy_data(excel_file,source_root,destination_root,categories)

How many subjects do we have in each group?

In [ ]:
data_root = "Data"
categories = ["AD", "MCI", "NC"]

for c in categories:
    path_train = os.path.join(data_root, 'train', c)
    path_test = os.path.join(data_root, 'test', c)

    num_train_files = len(os.listdir(path_train))
    print(f"{c} train: {num_train_files} files")

    num_test_files = len(os.listdir(path_test))
    print(f"{c} test: {num_test_files} files")


Classification model: ResNet3d

ResNet50 is originally designed for 2D images, so modifying it for 3D MRI may not be optimal.
A better alternative would be ResNet-3D variants, like **ResNet18-3D** or **ResNet50-3D** from torchvision.models.video.

Image Dimensions (X, Y, Z): 79 x 95 x 79
Series Length:  (1 volume per scan == static images)
Voxel Dimensions (X, Y, Z): 2 x 2 x 2 Millimeters (standard MNI space)

In [ ]:
# Set random seed for reproducibility
torch.manual_seed(42)
torch.cuda.manual_seed(42)

# Load pretrained 3D ResNet (r3d_18)
resnet3d = models.r3d_18(weights=R3D_18_Weights.DEFAULT).to(device)

# Freeze all layers for transfer learning (do this first!)
for param in resnet3d.parameters():
    param.requires_grad = False

# Modify the first convolution layer to accept 1-channel 3D MRI input
resnet3d.stem[0] = nn.Conv3d(
    in_channels=1,  # Change to 1 channel for grayscale MRI images
    out_channels=64,  # Keeping the same output channels as the original model
    kernel_size=(7, 7, 7),  # The size of the 3D convolution filter
    stride=(2, 2, 2),  # reducing the spatial resolution by half at each step
    padding=(3, 3, 3),  # Adds zero-padding around the input before applying the convolution
    bias=False  # Whether the layer should learn a bias term (default = False)
).to(device)

# Modify the final fully connected layer (fc) for 3-class classification (AD, MCI, NC)
resnet3d.fc = nn.Sequential(
    nn.Dropout(p=0.2),
    nn.Linear(in_features=512, out_features=3)  # 3-class output
).to(device)

'''
Output size = ((input size + 2*padding size - kernel size)stride size) - 1
'''

# Print model summary
summary(model=resnet3d,
        input_size=(16, 1, 79, 95, 79), # (batch_size, channels, depth, height, width)
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
)

Data loader: Prepare the data for model training and testing

In [ ]:
# Define the batch size
batch_size = 16

# Define the transformation pipeline
transform = transforms.Compose([
    # Resize(spatial_size=(64, 64, 64)),  # Resize to 64x64x64 for 3D images
    # transforms.ToTensor(),  # Convert the image to a tensor
    transforms.Normalize(mean=[0.5], std=[0.5])  # Normalization parameters
])

# Data loader
train_data_path = os.path.join("Data","train")
test_data_path = os.path.join("Data","test")

train_dataloader_pretrained, test_dataloader_pretrained, class_names = data_setup.create_dataloaders(
    train_dir=train_data_path, 
    test_dir=test_data_path,
    transform=transform,
    batch_size=batch_size,
)

print(' ')
print(f"Class names: {class_names}")
print(f"Number of classes: {len(class_names)}")

print(' ')
print("Number of training data: ", len(train_dataloader_pretrained) * batch_size)
print("Number of testing data: ", len(test_dataloader_pretrained) * batch_size)


image_batch, label_batch = next(iter(train_dataloader_pretrained))
print(image_batch.shape, label_batch.shape)

# image, label = image_batch[0], label_batch[0]
# print(image.shape, label)

# plt.imshow(image.permute(1, 2, 0))
# plt.title(class_names[label])
# plt.axis(False)

In [ ]:
# Check the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Working on device: {device}")

# Create optimizer and loss function
optimizer = torch.optim.Adam(params=resnet3d.parameters(), lr=0.01, weight_decay=1e-3)
loss_fn = torch.nn.CrossEntropyLoss()

# Train the classifier head of the pretrained ViT feature extractor model
torch.manual_seed(42)
torch.cuda.manual_seed(42)

pretrained_RN_18_results = engine.train(
    model=resnet3d,
    train_dataloader=train_dataloader_pretrained,
    test_dataloader=test_dataloader_pretrained,
    optimizer=optimizer,
    loss_fn=loss_fn,
    epochs=5,
    device=device
)
